# 01 importing libraries

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
# importing the  data frame adjusted earlier: ords_prods_merge_updated.pkl
ords_prods_merge = pd.read_pickle('ords_prods_merge_updated.pkl')

In [5]:
# double check the df's dimensions
ords_prods_merge.shape

(32404859, 20)

In [6]:
df = ords_prods_merge

# 02 calculating aggregated mean column for the whole data set

In [7]:
df.groupby('department_id').agg({'order_number' : ['mean', 'median']})

order_number       
                      mean median
department_id                    
1                15.457838   10.0
2                17.277920   11.0
3                17.170395   11.0
4                17.811403   11.0
5                15.215751    9.0
6                16.439806   10.0
7                17.225802   11.0
8                15.340650   10.0
9                15.895474   10.0
10               20.197148   14.0
11               16.170638   10.0
12               15.887671   10.0
13               16.583536   10.0
14               16.773669   11.0
15               16.165037   10.0
16               17.665606   11.0
17               15.694469   10.0
18               19.310397   13.0
19               17.177343   11.0
20               16.473447   10.0
21               22.902379   16.0

#03_what's the difference btw aggregated mean in the whole data set and sample data set:
There is not so much difference as I compared both answers. Most of them in this recent calculation are 1 unit higher since it is on the whole data, except department_id: 20. The mean was 17.138607 in the first one million rows while in the whole data set it is 16.473447. I think it shows a right skew in this group of data (compared to the median).

# 04 create a loyalty flag for existing customers using the transform() and loc() functions.

In [8]:
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

/tmp/ipykernel_25088/3693603773.py:1: FutureWarning: The provided callable <function max at 0x7825721e9620> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)


In [9]:
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [10]:
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'


In [11]:
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# check the frequency of the flags
df['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [13]:
# we can check several columns without the need of using the head function as a whole
df[['user_id', 'loyalty_flag', 'order_number']].head(35)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,1
2,1,New customer,1
3,1,New customer,1
4,1,New customer,1
5,1,New customer,2
6,1,New customer,2
7,1,New customer,2
8,1,New customer,2
9,1,New customer,2


# 05 Identifying any difference between the spending habits of the three types of customers 

In [14]:
df.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The differences in proces show that loyal customers purchase cheaper products, as their mean purchase price is the lowest among the three customer types, while new customers choose higher prices, demonstrated by highest mean number (13,30). 

# 06 making spender flag

In [15]:
# first, let's check on which column should I conduct aggregation
df.head (5)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,merge_source,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2539329,1,prior,1,2,8,NaN,14084,2,0,...,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
2,2539329,1,prior,1,2,8,NaN,12427,3,0,...,23,19,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer
3,2539329,1,prior,1,2,8,NaN,26088,4,0,...,23,19,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer
4,2539329,1,prior,1,2,8,NaN,26405,5,0,...,54,17,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer


In [16]:
# The last colun, called avg_product_price is our tool to conduct the aggregation
df['avg_product_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)

/tmp/ipykernel_25088/320766942.py:2: FutureWarning: The provided callable <function mean at 0x7825721ea020> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df['avg_product_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
df[['user_id', 'avg_product_price']].head(15)

,user_id,avg_product_price
0,1,6.367797
1,1,6.367797
2,1,6.367797
3,1,6.367797
4,1,6.367797
5,1,6.367797
6,1,6.367797
7,1,6.367797
8,1,6.367797
9,1,6.367797


In [18]:
# setting the criteria for different spenders' flags
df.loc[df['avg_product_price'] >= 10, 'spending_flag'] = 'High spender'


In [19]:
df.loc[df['avg_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [20]:
# now check the frequency of the spending_flag column
df['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

# 07 make an Order Freuency flag 

In [21]:
# setting the aggregation through transform function
df['median_days_of_orders'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/tmp/ipykernel_25088/1673310048.py:2: FutureWarning: The provided callable <function median at 0x7825650ce480> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df['median_days_of_orders'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [22]:
# setting the criteria
df.loc[df['median_days_of_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'


In [23]:
df.loc[(df['median_days_of_orders'] <= 20) & (df['median_days_of_orders'] > 10), 'order_frequency_flag'] = 'Regular customer'


In [24]:
df.loc[df['median_days_of_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [25]:
# checking the value count
df['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [26]:
# final check on the adjustments
df[['user_id', 'days_since_prior_order', 'median_days_of_orders', 'order_frequency_flag']].head(20)

,user_id,days_since_prior_order,median_days_of_orders,order_frequency_flag
0,1,NaN,20.5,Non-frequent customer
1,1,NaN,20.5,Non-frequent customer
2,1,NaN,20.5,Non-frequent customer
3,1,NaN,20.5,Non-frequent customer
4,1,NaN,20.5,Non-frequent customer
5,1,15.0,20.5,Non-frequent customer
6,1,15.0,20.5,Non-frequent customer
7,1,15.0,20.5,Non-frequent customer
8,1,15.0,20.5,Non-frequent customer
9,1,15.0,20.5,Non-frequent customer


In [27]:
# exporting the data frame
# df.to_pickle('orders_products_merged_aggre_grouped_final.pkl')

In [28]:
import pyarrow
print(pyarrow.__version__)

22.0.0


In [29]:
# exporting the data frame to parquet to save more memory
df.to_parquet("orders_products_merged_aggre_grouped_final.parquet", compression = "snappy")